In [1]:
!pip uninstall numpy
!pip install numpy==1.26.4

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.0.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-99b71e71.so
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 68.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which i

In [2]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2463302 sha256=302cf3ac3ffbb7405eee2ba3485ddb7bb45212b0f28b1b5e4976da51330787e0
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
import pandas as pd
from collections import defaultdict
from surprise import Reader, Dataset, SVD, dump, NMF, accuracy, SlopeOne
from surprise.model_selection import cross_validate, train_test_split, KFold, GridSearchCV
# from sklearn.model_selection import train_test_split

In [10]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


In [2]:
ratings= pd.read_csv('/content/drive/MyDrive/Kinopoisk/rating_df.csv', usecols=['userId', 'movieId', 'rating'])

In [3]:
reader = Reader(rating_scale=(1,5))

In [4]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [5]:
del ratings

In [6]:
trainset, testset = train_test_split(data, test_size=0.2)

In [7]:
del data

In [8]:
slopeone=SlopeOne()
slopeone.fit(trainset)
del trainset

predictions = slopeone.test(testset)

In [15]:
print(f"SlopOne model's RMSE {accuracy.rmse(predictions)}")
print(f"SlopOne model's MAE {accuracy.mae(predictions)}")
print(f"SlopOne model's FCP {accuracy.fcp(predictions)}")

RMSE: 0.8766
SlopOne model's RMSE 0.8765935298566585
MAE:  0.6675
SlopOne model's MAE 0.6675032732572561
FCP:  0.6745
SlopOne model's FCP 0.6744658099851403


In [13]:
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=3.5)

# Precision and recall can then be averaged over all users
print(f"SlopeOne model's precision at 10 {sum(prec for prec in precisions.values()) / len(precisions)}")
print(f"SlopeOne model's recall at 10 {sum(rec for rec in recalls.values()) / len(recalls)}")

SlopeOne model's precision at 10 0.7348297656007476
SlopeOne model's recall at 10 0.6256987486679836


In [25]:
ratings= pd.read_csv('/content/drive/MyDrive/Kinopoisk/rating_df.csv', usecols=['userId', 'movieId', 'rating'])
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [26]:
trainset = data.build_full_trainset()
slopeone=SlopeOne()
slopeone.fit(trainset)


In [27]:
dump.dump('/content/drive/MyDrive/Kinopoisk/slopone_dump_file1', algo=slopeone)